# Feat 2 with FSL 
## Setup and QC 

In [7]:
import glob
import pandas as pd
import os
from shutil import rmtree
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)

import subprocess

In [8]:

DER_DIR = "/projects/niblab/bids_projects/Experiments/Bevel/derivatives"
SUB_DIR = "/projects/niblab/bids_projects/Experiments/Bevel/derivatives/sub-*"
SUB_DIRS = sorted(glob.glob(SUB_DIR))



### Write model 2 files  
Model template file: [design2.fsf]()

In [4]:
def writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf):
    outpath = os.path.join(DER_DIR,subID, "func/Analysis/feat2/{}_exp2.0".format(subID) )
    print(">>> SETTING DESIGN OUTPATH: ", outpath)
    tempfsf = tempfsf.replace("OUTPUT", outpath)
    #print(FEATS)
    i = 1
    for run_path in feat1_folders:
        #run = run_path.split("/")[-1].split(".")[0].split("_")[1]
        run = "run%s"%i
        #print("> %s : %s"%(run, run_path))
        tempfsf = tempfsf.replace(run, run_path)
        i += 1
    #print(tempfsf)
    OUTFILE_PATH = os.path.join(DER_DIR,subID, "func/Analysis/feat2/%s_design_exp2.0.fsf"%subID)
    print("OUTFILE ------------------------>>>> ", OUTFILE_PATH)
    with open(OUTFILE_PATH, "w") as outfile:
        outfile.write(tempfsf)
    outfile.close()


In [ ]:
for x in SUB_DIRS:
    good_runs = 0
    subID =  x.split("/")[-1]
    feat1_folders= glob.glob(os.path.join(DER_DIR, "{}/func/Analysis/feat1/*expanded2.0.feat".format(x)))
    for folder in feat1_folders:
        zstats=glob.glob(os.path.join(folder, "stats/cope*.nii.gz"))
        #print(zstats)
        if not zstats:
            print("BAD FOLDER:\n", folder)
            #rmtree(folder)
        else:
            good_runs += 1
    
        
    # start analysis
    
    ### check for existence of feat2 directory
    FEAT2_DIR = os.path.join(subID, "func/Analysis/feat2")
    if os.path.exists(FEAT2_DIR):
        pass
    else:
        os.makedirs(FEAT2_DIR)
    #print("> STARTING PROGRAM......")
    #print("----------------------->>>>SUBJECT: ",sub)

    if good_runs == 4:
        with open(os.path.join(DER_DIR, "design_files/design2.fsf"), 'r') as infile:
            tempfsf=infile.read()
            writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf)
            infile.close()
        
    elif good_runs == 3:
        with open(os.path.join(DER_DIR, "design_files/design2_3.fsf"), 'r') as infile:
            tempfsf=infile.read()
            writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf)
            infile.close()
    elif good_runs == 2:
        with open(os.path.join(DER_DIR, "design_files/design2_2.fsf"), 'r') as infile:
            tempfsf=infile.read()
            writeDesignFile(DER_DIR, subID, feat1_folders,tempfsf)
            infile.close()
    else:
        pass
    
    

### Run slurm job

In [18]:
bash_file = os.path.join('/projects/niblab/bids_projects/Experiments/Bevel/derivatives/code', 'feat2_bevel.job')

sub_ct = len(SUB_DIRS)
#print(sub_ct)
x=1
shell_cmd = "sbatch --array={}-{}%{} {}".format(1,90, sub_ct, bash_file)
print(shell_cmd)
os.system(shell_cmd)


#subject_set = [x.split("/")[-1].split("-")[1].lstrip("0") for x in subject_set]


# by subject 
#for sub_num in sorted(subject_set):
    #shell_cmd = "sbatch --array={}%1 {}".format(sub_num, bash_file)
    #os.system(shell_cmd)
    #print(shell_cmd)
    

sbatch --array=1-90%85 /projects/niblab/bids_projects/Experiments/Bevel/derivatives/code/feat2_bevel.job


0

In [23]:
!squeue -u nbytes

             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)


## Quality Check

In [3]:

from nipype.interfaces.fsl import ImageStats


200330-16:02:02,478 nipype.utils INFO:
	 Running nipype version 1.3.1 (latest: 1.4.2)


In [12]:
mean_func_ct=0
qc_dict={}
lvl2_dict={}
subjects_with_0s = []
for sub_dir in SUB_DIRS:
    sub_id=sub_dir.split("/")[-1]
    
    if sub_id not in qc_dict:
        qc_dict[sub_id] = {}
        
    if sub_id not in lvl2_dict:
        lvl2_dict[sub_id] = {}
        
    feat2_folder=os.path.join(sub_dir, 'func/Analysis/feat2/{}_exp2.0.gfeat'.format(sub_id))
    #print(feat2_folder)
    
    
    mean_func = os.path.join(feat2_folder,'mean_func.nii.gz')
    cope_folders= glob.glob(os.path.join(feat2_folder, "*.feat"))
    cope_folder_ct = len(cope_folders)
    zstat_files = glob.glob(os.path.join(feat2_folder, "*.feat/stats/zstat1.nii.gz"))
    zstat_file_ct = len(zstat_files)
    
    qc_dict[sub_id]["zstat_file_ct"] = zstat_file_ct
    qc_dict[sub_id]["cope_folder_ct"] = cope_folder_ct
    
    
    ct=0
    for zstat in sorted(zstat_files):
        cope=zstat.split("/")[-3].split(".")[0]
        #print(cope)
        ct+=1
        #shell_cmd = "cluster -i {} -t 0 > temp_file.txt".format(zstat)
        #fslstats_cmd = "fslstats {} -R".format(zstat)
        #try:
        #stats = ImageStats(in_file=zstat, op_string= '-R')#, terminal_output='')
        #stats.cmdline ==
        cmd='fslstats %s -R'%zstat
        #voxel_intensity = stats.run()
        #output = list(stats.aggregate_outputs()
        process = subprocess.Popen(cmd.split(), stdout=subprocess.PIPE)
        output, error = process.communicate()              

        output=output.decode("utf-8")
        output=output.strip("\n")
        lvl2_dict[sub_id][cope] = output
        if "0.0" in output:
            if sub_id not in subjects_with_0s:
                subjects_with_0s.append(sub_id)
            #print(fslstats_cmd)
            #print(os.system(fslstats_cmd))
            #cluster = check_output(['fslstats', zstat, '-R'], shell=True)
            #temp=pd.read_csv("temp_file.txt", sep='\t')
            #max_vox=temp["MAX"].max()
            #print(cluster)
        #except:
        #max_vox=None
        #print("fail")
        
        
        #qc_dict[sub_id]["zstat%i"%ct] = max_vox
        #print(ct, zstat)
    
    if os.path.exists(mean_func):
        mean_func_ct+=1
        qc_dict[sub_id]["mean_func"] = 1

    else:
        pass
 

In [26]:
print("subjects with 0 signal in copes: ", len(subjects_with_0s))
lvl3_subs = [x.split("/")[-1] for x in SUB_DIRS if x.split("/")[-1] not in subjects_with_0s]
print("perfect subjects:\t", len(lvl3_subs))
print("subjects: ", lvl3_subs)

subjects with 0 signal in copes:  35
perfect subjects:	 50
subjects:  ['sub-001', 'sub-004', 'sub-005', 'sub-007', 'sub-009', 'sub-010', 'sub-011', 'sub-012', 'sub-013', 'sub-015', 'sub-016', 'sub-017', 'sub-018', 'sub-020', 'sub-021', 'sub-022', 'sub-025', 'sub-027', 'sub-028', 'sub-029', 'sub-030', 'sub-032', 'sub-038', 'sub-040', 'sub-041', 'sub-042', 'sub-043', 'sub-044', 'sub-048', 'sub-053', 'sub-054', 'sub-056', 'sub-057', 'sub-059', 'sub-060', 'sub-062', 'sub-063', 'sub-064', 'sub-066', 'sub-068', 'sub-069', 'sub-071', 'sub-072', 'sub-075', 'sub-076', 'sub-081', 'sub-084', 'sub-085', 'sub-088', 'sub-089']


In [164]:
lvl2_cope_df.head()

,cope1,cope10,cope11,cope12,cope13,cope14,cope15,cope16,cope17,cope18,cope19,cope2,cope20,cope21,cope22,cope23,cope3,cope4,cope5,cope6,cope7,cope8,cope9
sub-001,-5.149089 5.400637,-4.510490 4.929140,-4.621045 3.823775,-5.049824 5.417955,-4.844965 5.470177,-5.470177 4.844965,-7.269985 5.062478,-5.062478 7.269916,-5.703035 5.045375,-3.916097 5.930358,-6.710282 4.420660,-5.400637 5.149089,-4.420660 6.710280,-4.974420 5.236734,-5.236734 4.974420,-5.257060 4.422542,-5.546316 5.001424,-5.001424 5.546316,-4.815015 5.144981,-4.704127 4.693715,-4.573304 5.131107,-5.131107 4.573304,-4.929140 4.510490
sub-002,-4.627014 4.598331,-4.823052 4.452289,-6.532806 5.661823,-6.651571 5.299552,-7.799837 6.376823,-6.376823 7.799837,-5.277029 5.446514,-5.446514 5.277029,-6.017691 8.069314,-5.067532 6.222901,-6.367537 6.872022,-4.598331 4.627014,-6.872013 6.367537,-11.021873 7.518949,-7.518949 11.021873,-8.179813 7.908150,0.000000 0.000000,0.000000 0.000000,-6.433298 6.086695,-6.722400 7.558539,-6.626678 7.793071,-7.793071 6.626678,-4.452289 4.823052
sub-003,-5.421859 7.118972,-5.682829 5.446904,-10.391096 9.563452,-6.904866 5.664588,-8.455349 6.262292,-6.262292 8.455349,-4.850310 4.892520,-4.892520 4.850310,-4.526897 5.060896,0.000000 0.000000,-5.168696 5.246500,-7.119060 5.421859,-5.246500 5.168696,-7.067524 4.884535,-4.884535 7.067547,-4.625580 4.654994,-5.811203 7.941285,-7.941285 5.811203,-7.561533 6.108395,-7.495983 5.405548,-6.880083 7.225046,-7.224934 6.880102,-5.446904 5.682829
sub-004,-8.833944 5.623909,-5.795523 4.151964,-7.338194 8.384481,-5.646645 4.260946,-7.283653 5.573559,-5.573559 7.283765,-4.667728 5.152958,-5.152958 4.667728,-7.433256 6.379904,-5.503895 5.503123,-5.762137 5.095201,-5.623909 8.833944,-5.095201 5.762137,-6.633983 4.872158,-4.872158 6.633975,-5.406233 6.653097,-7.126720 6.445814,-6.445815 7.126756,-6.688675 10.279569,-5.774366 7.662568,-6.225459 5.732070,-5.732070 6.225459,-4.151964 5.795523
sub-005,-9.408529 6.345373,-4.935070 10.571926,-4.649796 8.081814,-4.667770 5.691463,-7.600784 4.283872,-4.283872 7.600784,-5.320257 5.388280,-5.388280 5.320257,-4.277063 6.016151,-5.593900 6.020192,-6.561638 5.748898,-6.345372 9.408529,-5.748898 6.561637,-6.051173 4.691402,-4.691402 6.051173,-4.337674 6.438028,-5.497678 6.257793,-6.257794 5.497678,-5.020307 5.528230,-6.014785 5.656413,-3.985212 4.653278,-4.653278 3.985212,-10.571926 4.935070


In [165]:
#def highlight_max(s):    
    #is_zero = s[0] == 
def color_negative_red(val):
    #print(val)
    color = 'red' if val == "0.000000 0.000000 " else 'black'
    #if color == 'red':
     #   print(color)
    #return ['background-color: red' if v else '' for v in is_zero]
    return 'color: %s' % color

In [166]:
#lvl2_cope_df.style.apply(highlight_max)
lvl2_cope_df = lvl2_cope_df.style.applymap(color_negative_red)

#lvl2_cope_df.to_excel("expanded2.0_lvl2_copes_voxel_check.csv", sep="\t")

#lvl2_cope_df.to_csv("expanded2.0_lvl2_copes_voxel_check.csv", sep="\t")
#lvl2_cope_df.loc[:,lvl2_cope_df.any() == "0.000000 0.000000"]

In [167]:
lvl2_cope_df

In [170]:
lvl2_cope_df.to_excel("expanded2.0_lvl2_copes_voxel_check.xlsx")

#lvl2_cope_df.("expanded2.0_lvl2_copes_voxel_check.csv", sep="\t")
#lvl2_cope_df.loc[:,lvl2_cope_df.any() == "0.000000 0.000000"]

In [6]:
df = pd.DataFrame(qc_dict).T
df.shape

(85, 3)

### Data count check

In [17]:
pd.set_option('display.max_rows', None)
print("\n\n# of good subjects ready for level 3: ", len(df[(df!=0).all(axis="columns")]))
display(df)



# of good subjects ready for level 3:  83


,zstat_file_ct,cope_folder_ct
sub-001,23,23
sub-002,23,23
sub-003,23,23
sub-004,23,23
sub-005,23,23
sub-006,23,23
sub-007,23,23
sub-009,23,23
sub-010,23,23
sub-011,23,23


### Subjects with missing data

In [12]:
df[(df==0).any(axis="columns")]

,zstat_file_ct,cope_folder_ct
sub-013,0,0
sub-053,0,0
